In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from MyDatasetLoader import MyDatasetLoader
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.model_selection import KFold
import torch
import numpy as np


def reduce_dim_pca(dataset,reduce_dim_to):
    pca = PCA(n_components=reduce_dim_to)
    x= pca.fit_transform(dataset[0].x)
   
    dataset.data.x =torch.tensor(x,dtype=torch.float32)
    return 
dataset=MyDatasetLoader(root='./cora')
#reduce_dim_pca(dataset,300)

train_x= dataset[0].x[dataset[0].train_mask].to('cpu').numpy()
train_y= dataset[0].y[dataset[0].train_mask].to('cpu').numpy()
test_x = dataset[0].x[dataset[0].test_mask].to('cpu').numpy()
test_y = dataset[0].y[dataset[0].test_mask].to('cpu').numpy()




Processing...
Done!


In [3]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
train_x= dataset[0].x[dataset[0].train_mask].to('cpu').numpy()
train_y= dataset[0].y[dataset[0].train_mask].to('cpu').numpy()
test_x = dataset[0].x[dataset[0].test_mask].to('cpu').numpy()
test_y = dataset[0].y[dataset[0].test_mask].to('cpu').numpy()

full_train_x=dataset[0].x.to('cpu').numpy()
full_train_y=dataset[0].y.to('cpu').numpy()


hot_full_train_y= ohe.fit_transform(full_train_y.reshape(-1, 1)).toarray()
hot_train_y = ohe.fit_transform(train_y.reshape(-1, 1)).toarray()
hot_test_y=  ohe.fit_transform(test_y.reshape(-1, 1)).toarray()



def convert_from_one_hot(pred_one_hot):
    pred = list()
    for i in range(len(pred_one_hot)):
        pred.append(np.argmax(pred_one_hot[i]))
    return pred



In [14]:
model = Sequential()
model.add(Dense(50, input_dim=len(train_x[0]), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_x, hot_train_y, epochs=20)

Epoch 1/20
51/51 [==============================] - 0s 902us/step - loss: 1.8904 - accuracy: 0.2560
Epoch 2/20
51/51 [==============================] - 0s 902us/step - loss: 1.7693 - accuracy: 0.3089
Epoch 3/20
51/51 [==============================] - 0s 853us/step - loss: 1.6654 - accuracy: 0.3311
Epoch 4/20
51/51 [==============================] - 0s 931us/step - loss: 1.5203 - accuracy: 0.3926
Epoch 5/20
51/51 [==============================] - 0s 902us/step - loss: 1.3236 - accuracy: 0.5077
Epoch 6/20
51/51 [==============================] - 0s 941us/step - loss: 1.1577 - accuracy: 0.5828
Epoch 7/20
51/51 [==============================] - 0s 941us/step - loss: 0.9949 - accuracy: 0.6418
Epoch 8/20
51/51 [==============================] - 0s 1ms/step - loss: 0.8838 - accuracy: 0.6702
Epoch 9/20
51/51 [==============================] - 0s 931us/step - loss: 0.7831 - accuracy: 0.7231
Epoch 10/20
51/51 [==============================] - 0s 971us/step - loss: 0.6803 - accuracy: 0.7508
E

In [17]:

pred=model.predict(test_x)

y_pred=convert_from_one_hot(pred)

print("Accuracy:",metrics.accuracy_score(test_y, y_pred))
print("f1:",metrics.f1_score(test_y, y_pred,average='macro'))
print("confusion matrix:\n",metrics.confusion_matrix(test_y, y_pred))

Accuracy: 0.7414589104339797
f1: 0.7126828581993351
confusion matrix:
 [[259   1  12  21  13  11   2]
 [  8  34   5   3  11   0   2]
 [ 11   1  64   7   3   6   4]
 [ 27   2   3 152   7   1   1]
 [ 26   6   3   8  81   7   8]
 [ 11   1   4   1   3 140   1]
 [  6   4   9   7   8   5  73]]


In [113]:

pred=model.predict(train_x)

y_pred=convert_from_one_hot(pred)

print("Accuracy:",metrics.accuracy_score(train_y, y_pred))
print("f1:",metrics.f1_score(train_y, y_pred,average='macro'))
print("confusion matrix:\n",metrics.confusion_matrix(train_y, y_pred))

Accuracy: 0.9981538461538462
f1: 0.9986674966941893
confusion matrix:
 [[506   0   0   1   0   0   0]
 [  0 107   0   0   0   0   0]
 [  0   0 130   0   0   0   0]
 [  1   0   0 263   0   0   0]
 [  1   0   0   0 193   0   0]
 [  0   0   0   0   0 248   0]
 [  0   0   0   0   0   0 175]]


In [18]:
from tensorflow.keras.losses import sparse_categorical_crossentropy
loss_function = sparse_categorical_crossentropy
kfold = KFold(n_splits=3, shuffle=True)
acc_per_fold = []
loss_per_fold = []
models=[]
fold_no=0

#train_x=full_train_x
#train_y=full_train_y
#hot_train_y=hot_full_train_y

for train, test in kfold.split(train_x, hot_train_y):
   
    
    model = Sequential()
    model.add(Dense(50, input_dim=len(train_x[0]), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7, activation='softmax'))

    model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy'])

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(train_x[train], train_y[train],
              batch_size=100,
              epochs=50,
             )
    scores = model.evaluate(train_x[test], train_y[test], verbose=0)
    models.append(model)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    pred=model.predict(train_x[test])
    y_pred=convert_from_one_hot(pred)
    print("Accuracy:",metrics.accuracy_score( train_y[test], y_pred))
    print("f1:",metrics.f1_score( train_y[test], y_pred,average='macro'))
    print("confusion matrix:\n",metrics.confusion_matrix(train_y[test], y_pred))
    
    
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1





------------------------------------------------------------------------
Training for fold 0 ...
Epoch 1/50
11/11 [==============================] - 0s 2ms/step - loss: 1.9019 - accuracy: 0.2881
Epoch 2/50
11/11 [==============================] - 0s 2ms/step - loss: 1.7980 - accuracy: 0.3102
Epoch 3/50
11/11 [==============================] - 0s 2ms/step - loss: 1.7001 - accuracy: 0.3102
Epoch 4/50
11/11 [==============================] - 0s 1ms/step - loss: 1.5785 - accuracy: 0.3232
Epoch 5/50
11/11 [==============================] - 0s 2ms/step - loss: 1.3947 - accuracy: 0.4349
Epoch 6/50
11/11 [==============================] - 0s 2ms/step - loss: 1.1859 - accuracy: 0.6159
Epoch 7/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9058 - accuracy: 0.7119
Epoch 8/50
11/11 [==============================] - 0s 2ms/step - loss: 0.6507 - accuracy: 0.8153
Epoch 9/50
11/11 [==============================] - 0s 2ms/step - loss: 0.4764 - accuracy: 0.8680
Epoch 10/50
11/11 [==

In [19]:

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')



------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 1.4687252044677734 - Accuracy: 74.16974306106567%
------------------------------------------------------------------------
> Fold 2 - Loss: 1.9200348854064941 - Accuracy: 70.47970294952393%
------------------------------------------------------------------------
> Fold 3 - Loss: 1.9082883596420288 - Accuracy: 68.2070255279541%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 70.95215717951457 (+- 2.4570863293063288)
> Loss: 1.7656828165054321
------------------------------------------------------------------------


In [20]:

for model,fold in zip(models,range(0,len(models))):
    print(f'-------fold number {fold}------')
    pred=model.predict(test_x)
    y_pred=convert_from_one_hot(pred)
    print("Accuracy:",metrics.accuracy_score(test_y, y_pred))
    print("f1:",metrics.f1_score(test_y, y_pred,average='macro'))
    print("confusion matrix:\n",metrics.confusion_matrix(test_y, y_pred))

-------fold number 0------
Accuracy: 0.7183748845798708
f1: 0.6953561374768409
confusion matrix:
 [[254   1   9  15  17  12  11]
 [  9  35   1   1   7   1   9]
 [ 10   1  60   9   4   7   5]
 [ 37   1   2 139   6   3   5]
 [ 29   6   1   7  79   5  12]
 [ 11   0   7   0   2 136   5]
 [ 11   5   1   7   7   6  75]]
-------fold number 1------
Accuracy: 0.7386888273314867
f1: 0.7124931988298286
confusion matrix:
 [[266   2   4  27  10   5   5]
 [ 13  32   0   1  11   2   4]
 [ 13   0  68   3   1   7   4]
 [ 32   4   3 145   5   2   2]
 [ 31   7   1  10  79   3   8]
 [ 11   1   3   1   2 137   6]
 [ 10   6   2   3   9   9  73]]
-------fold number 2------
Accuracy: 0.7386888273314867
f1: 0.7122533527952767
confusion matrix:
 [[265   1   6  18  14   9   6]
 [  3  35   2   4  11   3   5]
 [ 12   0  63   3   5   6   7]
 [ 31   4   1 146   6   3   2]
 [ 21   5   6   6  81   7  13]
 [ 16   1   5   1   3 131   4]
 [  7   2   2   4  10   8  79]]
